In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
family_group = "3.Satyrine"
annotated_genome_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Blast_result"
species_list = os.listdir(annotated_genome_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
# species_list = ["Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Limenitis_camilla"]
# species_list = ["Argynnis_bischoffii_washingtonia","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Bicyclus_anynana","Coenonympha_glycerion","Elymnias_hypermnestra","Erebia_aethiops","Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Vanessa_tameamea"]
for species in species_list:
    print(species)
    blast_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Blast_result"
    
    list_of_files = os.listdir(f"{blast_location}/{species}")
    query_species_list = []
    coordinate_detail_dictionary = {}
    coordinate_info_dictionary = {}
    scaffold = ''
    for file in list_of_files:
        if file.endswith(".csv") and len(file.split("_")) >= 5 and not(file.endswith("_old.csv")):
#             print(file)
            if "." in file[:-4]:
                query_species = f"{file.split('_')[-2].split('.')[1]}_{file.split('_')[-1].split('.')[0]}"
            else:
                query_species = f"{file.split('_')[-2]}_{file.split('_')[-1].split('.')[0]}"
            query_species_list.append(query_species)
            coordinate_detail_dictionary[query_species] = {}
            with open(f"{blast_location}/{species}/{file}", 'r') as coordinate_file_open:
                coordinate_file_lines_list  =  coordinate_file_open.readlines()
            if scaffold == '':
                scaffold = coordinate_file_lines_list[1].split(",")[1]
    
            for i in range(1,len(coordinate_file_lines_list)):
                
                coordinate_file_lines_split = coordinate_file_lines_list[i].split(",")
                start,stop = coordinate_file_lines_split[2],coordinate_file_lines_split[3]
                exon = f"Exon_{i}"
                if exon not in coordinate_info_dictionary:
                    coordinate_info_dictionary[exon] = {}
                coordinate_info_dictionary[exon][query_species] = [start,stop]
                coordinate_detail_dictionary[query_species][exon] = coordinate_file_lines_list[i]
            
            # break
            # print(query_species)
            # print(file)
    # print(coordinate_info_dictionary)
    # assert False
    output = coordinate_file_lines_list[0]
    for exon,species_specific_coordinates in coordinate_info_dictionary.items():
        # print(exon,species_specific_coordinates)
        coordinates_list = []
        previous_coordinates = ""
        match = 1
        for query_species, coordinates in coordinate_info_dictionary[exon].items():
            coordinates_list.append([query_species,coordinates])
            if previous_coordinates != '':
                if coordinates != previous_coordinates:
                    match = 0
                    
            previous_coordinates = coordinates
       
        
        
        if match == 0:
            print(exon,"mismatch")
            print(coordinates_list)
                
            while True:
                choice = input("Proceed?")
                if choice[0].lower() == "y" or choice[0].lower() == "n":
                    break
            
            while True:
                if choice[0].lower() == "y":
                    alignment_detail_dictionary = {}
                    for query_species_name in query_species_list:
                        list_of_folders  = os.listdir(f"{blast_location}/{species}")
                        for folders in list_of_folders:
                            if folders.endswith(query_species_name):
                                # print()
                                
                                with open(f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query.txt", 'r') as query_file_open:
                                    # alignment_detail = query_file_open.readlines()[0]
                                    alignment_detail = query_file_open.readlines()[0].split("set")[1][:-1]
                                    alignment_detail_dictionary[query_species_name] = alignment_detail
                                with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/{exon}_translated_genomic_sequence_{alignment_detail}.fa.hat2", 'r') as alignment_score_file:
                                    distance_score = float(alignment_score_file.readlines()[-1].rstrip())
                                try:
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query_spliced.txt", 'r') as new_query_file:
                                        new_query = (new_query_file.readlines()[-1].rstrip())
                                except:
                                    new_query = "Issue with query"
                                print(query_species_name,exon, alignment_detail, distance_score,f"\n{new_query}")
                    while True:            
                        check_alignment = input("Check Alignment??")
                        if check_alignment[0].lower() == "y" or check_alignment[0].lower() == "n":
                            break
                    if check_alignment[0].lower() == "y":
                        for query_species_name in query_species_list:
                            list_of_folders  = os.listdir(f"{blast_location}/{species}")
                            for folders in list_of_folders:
                                if folders.endswith(query_species_name):
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/alignment_clustal_{exon}_translated_genomic_sequence_{alignment_detail_dictionary[query_species_name]}.fa.txt", 'r') as clustal_alignment_file:
                                        clustal_alignment = clustal_alignment_file.readlines()
                                    print("".join(clustal_alignment))
                                    
                                # assert False
                    while True:
                        try:
                            species_number = (input(f"Choose Species 1 - {len(query_species_list)}"))
                            
                            
                            if species_number[0].lower() == 'n':
                                output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                                break 
                            output += coordinate_detail_dictionary[query_species_list[int(species_number)-1]][exon]
                            break
                        except:
                            print("Retry")
                    break
                elif choice[0].lower() == "n":
                    output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                    break
                
                
        else:
            print(exon, "Match")
            print(coordinates_list)
            output += coordinate_detail_dictionary[query_species_list[0]][exon]
        
       

        # assert False
    # print(coordinate_detail_dictionary)
    print(output)
    with open(f"{blast_location}/{species}/final_coordinates.csv", "w") as out_file:
        out_file.write(output)
    
    # 'Coenonympha_glycerion,OY979626.1,7015875,7015963,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29\n'

Coenonympha_glycerion
Exon_1 Match
[['Bicyclus_anynana', ['10535487', '10535583']], ['Maniola_jurtina', ['10535487', '10535583']], ['Pararge_aegeria', ['10535487', '10535583']]]
Exon_2 Match
[['Bicyclus_anynana', ['10533898', '10534448']], ['Maniola_jurtina', ['10533898', '10534448']], ['Pararge_aegeria', ['10533898', '10534448']]]
Exon_3 Match
[['Bicyclus_anynana', ['10533204', '10533357']], ['Maniola_jurtina', ['10533204', '10533357']], ['Pararge_aegeria', ['10533204', '10533357']]]
Exon_4 Match
[['Bicyclus_anynana', ['10532545', '10532789']], ['Maniola_jurtina', ['10532545', '10532789']], ['Pararge_aegeria', ['10532545', '10532789']]]
Exon_5 mismatch
[['Bicyclus_anynana', ['10532030', '10532313']], ['Maniola_jurtina', ['10532030', '10532403']], ['Pararge_aegeria', ['10532030', '10532313']]]
Proceed?y
Bicyclus_anynana Exon_5 15_frame0 0.673 
ENKNSKEVPKNTTEATQKNVKKNIVSETSDCGYGTLIENQESMSTSSNEDELPSKKPVHKKPHKPKQKVNNKQRPSVTLQERRRKKIVKRGKAN
Maniola_jurtina Exon_5 15_frame0 0.511 
AVVFFLFLN

Exon_1 Match
[['Bicyclus_anynana', ['5063241', '5063337']], ['Maniola_jurtina', ['5063241', '5063337']], ['Pararge_aegeria', ['5063241', '5063337']]]
Exon_2 Match
[['Bicyclus_anynana', ['5062617', '5063167']], ['Maniola_jurtina', ['5062617', '5063167']], ['Pararge_aegeria', ['5062617', '5063167']]]
Exon_3 Match
[['Bicyclus_anynana', ['5061538', '5061691']], ['Maniola_jurtina', ['5061538', '5061691']], ['Pararge_aegeria', ['5061538', '5061691']]]
Exon_4 Match
[['Bicyclus_anynana', ['5060037', '5060269']], ['Maniola_jurtina', ['5060037', '5060269']], ['Pararge_aegeria', ['5060037', '5060269']]]
Exon_5 Match
[['Bicyclus_anynana', ['5059457', '5059737']], ['Maniola_jurtina', ['5059457', '5059737']], ['Pararge_aegeria', ['5059457', '5059737']]]
Exon_6 Match
[['Bicyclus_anynana', ['5058769', '5059129']], ['Maniola_jurtina', ['5058769', '5059129']], ['Pararge_aegeria', ['5058769', '5059129']]]
Exon_7 Match
[['Bicyclus_anynana', ['5057428', '5057619']], ['Maniola_jurtina', ['5057428', '5057619

Exon_1 Match
[['Bicyclus_anynana', ['11529184', '11529283']], ['Maniola_jurtina', ['11529184', '11529283']], ['Pararge_aegeria', ['11529184', '11529283']]]
Exon_2 Match
[['Bicyclus_anynana', ['11529385', '11529935']], ['Maniola_jurtina', ['11529385', '11529935']], ['Pararge_aegeria', ['11529385', '11529935']]]
Exon_3 Match
[['Bicyclus_anynana', ['11530528', '11530681']], ['Maniola_jurtina', ['11530528', '11530681']], ['Pararge_aegeria', ['11530528', '11530681']]]
Exon_4 Match
[['Bicyclus_anynana', ['11533418', '11533665']], ['Maniola_jurtina', ['11533418', '11533665']], ['Pararge_aegeria', ['11533418', '11533665']]]
Exon_5 Match
[['Bicyclus_anynana', ['11534353', '11534633']], ['Maniola_jurtina', ['11534353', '11534633']], ['Pararge_aegeria', ['11534353', '11534633']]]
Exon_6 Match
[['Bicyclus_anynana', ['11535432', '11535792']], ['Maniola_jurtina', ['11535432', '11535792']], ['Pararge_aegeria', ['11535432', '11535792']]]
Exon_7 Match
[['Bicyclus_anynana', ['11536072', '11536263']], ['

Exon_1 Match
[['Bicyclus_anynana', ['9161789', '9161885']], ['Maniola_jurtina', ['9161789', '9161885']], ['Pararge_aegeria', ['9161789', '9161885']]]
Exon_2 Match
[['Bicyclus_anynana', ['9161975', '9162525']], ['Maniola_jurtina', ['9161975', '9162525']], ['Pararge_aegeria', ['9161975', '9162525']]]
Exon_3 Match
[['Bicyclus_anynana', ['9165408', '9165561']], ['Maniola_jurtina', ['9165408', '9165561']], ['Pararge_aegeria', ['9165408', '9165561']]]
Exon_4 Match
[['Bicyclus_anynana', ['9166845', '9167086']], ['Maniola_jurtina', ['9166845', '9167086']], ['Pararge_aegeria', ['9166845', '9167086']]]
Exon_5 Match
[['Bicyclus_anynana', ['9167769', '9168046']], ['Maniola_jurtina', ['9167769', '9168046']], ['Pararge_aegeria', ['9167769', '9168046']]]
Exon_6 Match
[['Bicyclus_anynana', ['9168763', '9169123']], ['Maniola_jurtina', ['9168763', '9169123']], ['Pararge_aegeria', ['9168763', '9169123']]]
Exon_7 Match
[['Bicyclus_anynana', ['9169287', '9169478']], ['Maniola_jurtina', ['9169287', '9169478

Exon_1 Match
[['Bicyclus_anynana', ['9419127', '9419226']], ['Maniola_jurtina', ['9419127', '9419226']], ['Pararge_aegeria', ['9419127', '9419226']]]
Exon_2 Match
[['Bicyclus_anynana', ['9418476', '9419026']], ['Maniola_jurtina', ['9418476', '9419026']], ['Pararge_aegeria', ['9418476', '9419026']]]
Exon_3 Match
[['Bicyclus_anynana', ['9418024', '9418177']], ['Maniola_jurtina', ['9418024', '9418177']], ['Pararge_aegeria', ['9418024', '9418177']]]
Exon_4 Match
[['Bicyclus_anynana', ['9417258', '9417499']], ['Maniola_jurtina', ['9417258', '9417499']], ['Pararge_aegeria', ['9417258', '9417499']]]
Exon_5 Match
[['Bicyclus_anynana', ['9416483', '9416763']], ['Maniola_jurtina', ['9416483', '9416763']], ['Pararge_aegeria', ['9416483', '9416763']]]
Exon_6 Match
[['Bicyclus_anynana', ['9416035', '9416395']], ['Maniola_jurtina', ['9416035', '9416395']], ['Pararge_aegeria', ['9416035', '9416395']]]
Exon_7 Match
[['Bicyclus_anynana', ['9415116', '9415307']], ['Maniola_jurtina', ['9415116', '9415307

Proceed?y
Bicyclus_anynana Exon_15 39_frame0 1.401 
SIIAEDEDAVMKPVEQKVLVSLAAAEPARGPPRDKLDQLDSDYCNN
Maniola_jurtina Exon_15 39_frame0 1.179 
SIIAEDEDAVMKPVEQKVLVSLAAAEPARGPPRDKLDQLDSDYCNN
Pararge_aegeria Exon_15 1_frame2 1.804 
QSSILNSAETRSVILPELSKGNVPTPMTDNLYQIHNKKQNPFASIVNFTP
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Bicyclus_anynan -------SIIAEDEDEVMQVADEELHVSLEAKPRPLVTPAGPVTLMMSTIMSVKQETTNF
set39_frame0    WWCPSVRSIIAEDEDAVMKPVEQKVLVSLAAAEPARGPPRD-------------------
                       ******** **: .:::: *** *   .  .* .                   

Bicyclus_anynan TIGQAESDYCNN
set39_frame0    KLDQLDSDYCNN
                .:.* :******

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Maniola_jurtina -------SIIAEDEDEVMKLEEESVRVSLEAVPPVRIATVLSGKQEPAGFSGQAESDYCN
set39_frame0    WWCPSVRSIIAEDEDAVMKPVEQKVLVSLAAAEPAR--------GPPRDKLDQLDSDYCN
                       ******** ***  *:.* *** *. *.*          * .  .* :*****

Maniola_jurtina N
set39_frame0    N
         

Exon_1 Match
[['Bicyclus_anynana', ['11821084', '11821180']], ['Maniola_jurtina', ['11821084', '11821180']], ['Pararge_aegeria', ['11821084', '11821180']]]
Exon_2 Match
[['Bicyclus_anynana', ['11820453', '11821003']], ['Maniola_jurtina', ['11820453', '11821003']], ['Pararge_aegeria', ['11820453', '11821003']]]
Exon_3 Match
[['Bicyclus_anynana', ['11820127', '11820280']], ['Maniola_jurtina', ['11820127', '11820280']], ['Pararge_aegeria', ['11820127', '11820280']]]
Exon_4 Match
[['Bicyclus_anynana', ['11816321', '11816562']], ['Maniola_jurtina', ['11816321', '11816562']], ['Pararge_aegeria', ['11816321', '11816562']]]
Exon_5 Match
[['Bicyclus_anynana', ['11815645', '11815919']], ['Maniola_jurtina', ['11815645', '11815919']], ['Pararge_aegeria', ['11815645', '11815919']]]
Exon_6 Match
[['Bicyclus_anynana', ['11814640', '11815000']], ['Maniola_jurtina', ['11814640', '11815000']], ['Pararge_aegeria', ['11814640', '11815000']]]
Exon_7 Match
[['Bicyclus_anynana', ['11814208', '11814399']], ['